In [1]:
import numpy as np
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp, Statevector
from tabulate import tabulate
from itertools import combinations
from qiskit.circuit.library import EfficientSU2
import math
import time
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.time_evolvers.variational import ImaginaryMcLachlanPrinciple
from qiskit.algorithms import TimeEvolutionProblem
from qiskit.algorithms import VarQITE
from qiskit.primitives import Estimator
from qiskit.quantum_info import Statevector
from qiskit.algorithms import SciPyImaginaryEvolver
import pylab
from qiskit.algorithms.gradients import ReverseEstimatorGradient, ReverseQGT
from qiskit.circuit.library import RYGate
import matplotlib.pyplot as plt

import os
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
from scipy.linalg import eigh, block_diag

In [2]:
'''Check the hamiltonian, changed the parameters for this file'''
def spinless_sub_dim(N,r) : 
    '''
    input : number of lattice sites (N), number of electrons (r) 
    output : dimension of this subspace 
    '''
    return math.comb(N,r)


def hubb0_model(N,r,e,t,U): 
    ''' Generalised Tight Binding (spinless) model for periodic boundary condition
    Input : Number of lattice sites (int), number of electrons(int), hopping constant (int/float), onsite energies (list), interaction term U (int)
    Output : Tight binding Hamiltonian, eigenvalues and eigenvectors of the matrix ''' 
    dim = spinless_sub_dim(N,r)
    #Special Cases
    if r==0 : 
        H = np.zeros(1)
        eigval = 0
        new_vec = [[1]]
    elif r==N : 
        H = [[sum(e)+N*U]]
        eigval = H[0]
        new_vec = [[1]]
    if N == 1 and r==1: 
        H = [e]
        eigval = H[0]
        new_vec = [[1]]
    else : 
        H = np.zeros((dim, dim))
        basis_set = spinless_basis(N,r)
        #H_diagonal, onsite energy terms
        n_diag = np.zeros(dim)
        for i in range(dim) : 
            for j in range(N) : 
                n_diag[i] += e[j]*basis_set[i][j]
        np.fill_diagonal(H,n_diag)
        #H_T Hopping terms 
        for basis_index,basis in enumerate(basis_set) : 
            for site in range(len(basis)) : 
                if basis[site] == False and basis[(site+1)%N] == True : 
                    new_state = basis.copy()
                    new_state[site] = True
                    new_state[(site+1)%N] = False 
                    for i in range(len(basis_set)) : 
                        if basis_set[i] == new_state: 
                            f_index = i
                    H[f_index][basis_index] +=t
                if N != 2 : 
                    if basis[site] == True and basis[(site+1)%N] == False : 
                        new_state = basis.copy()
                        new_state[site] = False
                        new_state[(site+1)%N] = True 
                        for i in range(len(basis_set)) : 
                            if basis_set[i] == new_state : 
                                f_index = i
                        H[f_index][basis_index] +=t 
        #H_U, interaction terms
        for basis_index,basis in enumerate(basis_set) : 
            for site in range(len(basis)) : 
                if basis[site] == True and basis[(site+1)%N] == True : 
                    H[basis_index][basis_index] +=U

        eigval,eigvec = np.linalg.eigh(H)
        new_vec = list(zip(*eigvec))                   
    return H,eigval,new_vec

def hubb0_full(N,e,t,U): 
    '''Full Block Diagonal Hamiltonian for some N length closed lattice '''
    H_sub_list = []
    for r in range(N+1) : 
        H_sub = hubb0_model(N,r,e,t,U)[0]
        H_sub_list.append(H_sub)
    H = block_diag(*H_sub_list) 
    return H

#JW functions

def hubb0_JW(N,e,t,U) : 
    strings = []
    opt = SparsePauliOp.from_sparse_list([("I", [0], 0)], num_qubits=N)  
    for k in range(N) : 
        a0='I'*N
        a1 = 'I'*(k)+'Z' +'I'*(N-k-1)

        b0='I'*N
        b0_list = list(b0)
        b0_list[k] = 'X'
        b0_list[(k+1)%N] = 'X'
        new_b0 = ''.join(b0_list)

        b1='I'*N
        b1_list = list(b0)
        b1_list[k] = 'Y'
        b1_list[(k+1)%N] = 'Y'
        new_b1 = ''.join(b1_list)
        
        c_base='I'*N

        c0 = c_base
        
        c1_list = list(c_base)
        c1_list[k] = 'I'
        c1_list[(k+1)%N] = 'Z'
        c1 = ''.join(c1_list)
        
        c2_list = list(c_base)
        c2_list[k] = 'Z'
        c2_list[(k+1)%N] = 'I'
        c2 = ''.join(c2_list)
        
        c3_list = list(c_base)
        c3_list[k] = 'Z'
        c3_list[(k+1)%N] = 'Z'
        c3 = ''.join(c3_list)
        T=t
        if N==2 : 
            T=t/2
        
        opt += SparsePauliOp.from_list([(a0, 0.5*e[k]), (a1, -0.5*e[k]),
                                        (new_b0, 0.5*T),(new_b1, 0.5*T),
                                        (c0, U*0.25),(c1, -0.25*U),(c2, -0.25*U),(c3,U*0.25)])
    return opt


def spinless_basis(N,r) : 
    basis_set = []
    lattice = list(range(N))
    places = list(combinations(lattice, r))
    for combination in places : 
        basis = [False] *N
        for index in combination : 
            basis[index] = True 
        basis_set.append(basis)
    return basis_set

def scs_param(n,l,var,param) : 

    circ = QuantumCircuit(n)
    circ.cx(-2,-1)
    circ.cry(param[var],-1,-2)
    var+=1
    circ.cx(-2,-1)

    for i in range(l-1) : 
        circ.cx(-3-i,-1)
        ccry = RYGate(param[var]).control(2,label=None)
        var+=1
        circ.append(ccry,[-1,-2-i,-3-i])
        circ.cx(-3-i,-1)
    return circ, var

def dicke_param(n,k) : 
    pairs = []
    for a in range(n,k,-1) : 
        pairs.append([a,k])
    for a in range(k,1,-1) : 
        pairs.append([a,a-1])

    num_angles = int(k*(n-k) + k*(k-1)/2)
    param = [Parameter(f"angle_{i+1}") for i in range(num_angles)]

    dk_circ = QuantumCircuit(n)
    for ind in range(k) : 
        dk_circ.x(-(1+ind))
    var=0
    for pair in pairs : 
        new_circ,new_var = scs_param(pair[0],pair[1],var,param)
        var = new_var
        dk_circ.append(new_circ, range(pair[0]))
    return dk_circ


In [3]:
backend = Aer.get_backend('statevector_simulator')

In [4]:
pdf_filename = os.path.join("U8.pdf")
pdf_pages = PdfPages(pdf_filename)
results = []

#N = 5

for N in range(2,9) : 
    for r in range(1,N) : 
        e=[0]*N
        t=1
        U=8
        H_op = hubb0_JW(N,e,t,U)
        
        H_class,e_tb, v_tb = hubb0_model(N,r,e,t,U) 
        eig_true = min(e_tb)
        
        e_jw, vec = np.linalg.eigh(hubb0_JW(N,e,t,U).to_matrix())
        v_jw = list(zip(*vec))
        state = v_jw[0]
        state_true = [np.round(x,3) for x in state]
        
        ansatz = dicke_param(N,r)
        init_param_values={}
        for i in range(len(ansatz.parameters)):
            init_param_values[ansatz.parameters[i]]=1

        exp_time = 15.0
        num_steps = 250

        aux_ops = [H_op]
        evolution_problem = TimeEvolutionProblem(H_op, exp_time, aux_operators=aux_ops)

        start_time = time.time()

        #var_principle = ImaginaryMcLachlanPrinciple()
        var_principle = ImaginaryMcLachlanPrinciple(qgt = ReverseQGT() , 
                                                    gradient = ReverseEstimatorGradient())

        evolution_problem = TimeEvolutionProblem(H_op, exp_time, aux_operators=aux_ops)
        var_qite = VarQITE(ansatz, init_param_values, var_principle, Estimator(),
                          num_timesteps=num_steps)
        evolution_result_eff = var_qite.evolve(evolution_problem)
        end_time = time.time()
        elapsed_time = end_time - start_time

        eff_circ = evolution_result_eff.evolved_state

        eff_job = execute(eff_circ, backend)
        eff_result = eff_job.result()
        eff_statevector = eff_result.get_statevector()

        sum_of_squares = (np.array(eff_statevector).conj() @ np.array(eff_statevector)).real
        norm_state = eff_statevector/np.sqrt(sum_of_squares)
        final_sv = [np.round(x,3) for x in np.asarray(norm_state)]

        overlap =np.dot(state,np.conj(norm_state))

        exp_q = np.array([ele[0][0] for ele in evolution_result_eff.observables])

        #Classical simulation
        start_class = time.time()
        init_state = Statevector(ansatz.assign_parameters(init_param_values))

        evolution_problem = TimeEvolutionProblem(H_op, exp_time, initial_state=init_state, 
                                                 aux_operators=aux_ops)
        exact_evol = SciPyImaginaryEvolver(num_timesteps=num_steps)
        sol = exact_evol.evolve(evolution_problem)
        end_class = time.time() 
        time_class = end_class - start_class
        
        exp_c = sol.observables[0][0].real
        #print("Exact lowest eigenvalue found : ",exact_h_exp_val[-1]) 

        times_q = evolution_result_eff.times
        times_c = sol.times

        pylab.plot(times_q, exp_q, label= "VarQITE")
        pylab.plot(times_c, exp_c , label= "ScipyEvolver")
        pylab.plot(times_q, [eig_true]*len(times_q),linestyle='--', label='True')
        pylab.xlabel("Time")
        pylab.ylabel(r"$\langle H \rangle$ (energy)")
        pylab.legend(loc="upper right");
        plt.title(f' N={N}, r={r}, steps = {num_steps} ')

        pdf_pages.savefig()
        plt.close()

        results.append({'N' : N, 'eig_q' : np.round(exp_q[-1],3), 'eig_c': np.round(exp_c[-1],3), 
                        'eig_true' : np.round(eig_true,3), 'state_overlap' : np.round(overlap,3),
                        'time_q' : np.round(elapsed_time,3), 'time_clas' : np.round(time_class,3), 
                        'state_est': final_sv, 'state_true': state_true} )
    
pdf_pages.close()
df = pd.DataFrame(results)   

In [5]:
print(df)
df.to_csv('U8.csv', index=False)

    N   eig_q  eig_c  eig_true  state_overlap    time_q  time_clas  \
0   2  -1.000 -1.000    -1.000   1.000+0.000j    14.079      0.531   
1   3  -1.000 -1.000    -1.000   0.583+0.000j    31.226      0.562   
2   3   7.000 -1.000     7.000   0.000+0.000j    54.539      0.562   
3   4  -2.000 -2.000    -2.000   1.000+0.000j    38.365      0.656   
4   4  -0.486 -0.899    -0.899   0.000+0.000j   115.165      0.699   
5   4  14.000 -2.000    14.000  -0.000+0.000j   184.767      0.761   
6   5  -1.618 -1.618    -1.618  -0.000+0.000j    58.332      0.917   
7   5  -1.479 -1.883    -1.883  -0.895+0.000j   225.949      0.951   
8   5   6.455 -1.882     6.117  -0.000+0.000j   399.907      1.102   
9   5  22.382 -1.883    22.382  -0.000+0.000j   577.053      1.109   
10  6  -1.631 -2.000    -2.000  -0.000+0.000j   127.193      1.578   
11  6  -2.249 -3.016    -3.016   0.761+0.000j   456.677      1.934   
12  6  -0.608 -0.897    -0.887  -0.000+0.000j   857.308      1.929   
13  6  13.658 -3.016